# Virtual Branching for Person Re-ID

In [ ]:
import numpy as np
np.set_printoptions(threshold=1000)

import matplotlib.pyplot as plt
import time
import os
%matplotlib inline

from keras.callbacks import LearningRateScheduler, History

Using TensorFlow backend.


In [ ]:
import sys
sys.path.insert(0, '../src/')
import evaluation,data, training, models

### Load Data

In [ ]:
# Batch parameters for batch-hard triplet loss
P_param = 5
K_param = 4

In [ ]:
train_dict, train_files = data.get_data('train')

In [ ]:
start = time.time()
batch = data.batch_generator(train_dict, P=P_param, K=K_param, preprocess=True,
                                    shape=(256,128)).next()
print time.time() - start

i = 0

plt.figure(figsize=(15,2))
for j in range(P_param*K_param):
    plt.subplot(1,20,j+1)
    im = batch[0][i][j].squeeze().astype(np.uint8)
#         print im.min(), im.max()
    plt.imshow(im)
    plt.axis('off')
plt.show()

### Define Learning Rate Schedule

In [ ]:
epochs = 10
steps_per_epoch = 100

lr = []
for era in range(1,21):
    for j in range(10):
        lr.append(training.step_decay_cont(epochs, era)(j))
plt.plot(np.arange(200), lr)

### Define Model

In [ ]:
model = models.DenseNetDrop(P_param=P_param, K_param=K_param, weights='imagenet')

In [ ]:
# During evaluation, weights from model are loaded onto model_eval
model_eval = models.DenseNetDrop()

In [ ]:
# model.summary()

In [ ]:
history = History()
for era in range(1,16):
    iterations = era * epochs * steps_per_epoch
    lrate = LearningRateScheduler(training.step_decay_cont(epochs, era))

    print 'era, ' + str(era)

    model.fit_generator(data.batch_generator(train_dict, P=P_param, K=K_param,
                            preprocess=True, shape=(256,128)),
                         steps_per_epoch=steps_per_epoch,
                         epochs=epochs,
                         callbacks=[lrate,history])

    path = training.save_weights(model, it=iterations, root='vbranch_P5K4')
    model_eval.set_weights(model.get_weights())

    print evaluation.get_score(model_eval, hist=history, inputs=1)

In [ ]:
for it in range(1000, 14000, 1000):
    model_eval.set_weights(np.load('../models/vbranch_P5K4_pool3/vbranch_P5K4_pool3_%d.npy' % it))
    print evaluation.get_score(model_eval, inputs=1)

vbranch_P5K4 <br>
1000: r1=0.49 <br>
2000: {'mAP': 0.36899999999999999, 'rank': {'r5': 0.7931, 'r1': 0.5799}, 'loss': []} <br>
3000: {'mAP': 0.35020000000000001, 'rank': {'r5': 0.7755, 'r1': 0.5647}, 'loss': []} <br>
12000: {'mAP': 0.5544, 'rank': {'r5': 0.9062, 'r1': 0.7402}, 'loss': []}